In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import pylast
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import json
import random
from sklearn.neighbors import KNeighborsClassifier
import math

In [2]:
playlist_features = []
valid_data_indices = []
for i in range(0, 100):
    feature_sing = pd.read_csv("./playlists/df_playlist_features_" + str(i) + ".csv")
    feature_sing = feature_sing.drop(columns=['track_name', 'artist', 'id','Unnamed: 0'])
    feature_sing = feature_sing.dropna()
#     print(feature_sing.tail())
    feature_sing = feature_sing.to_numpy()
    num_songs = feature_sing.shape[0]
    if (num_songs >= 30):
        valid_data_indices.append(i)
        playlist_features.append(feature_sing[:30, ].reshape((30, 13, 1)))
playlist_features = np.array(playlist_features)
print(len(valid_data_indices))
playlist_features.shape

66


(66, 30, 13, 1)

In [3]:
# standardize features
playlist_features_mean = np.mean(playlist_features, axis=(0, 1), keepdims = True)
playlist_features_std = np.std(playlist_features, axis=(0, 1), keepdims = True)
playlist_features = (playlist_features - playlist_features_mean) / playlist_features_std
# playlist_features

In [4]:
# playlist_features_mean = np.mean(playlist_features, axis=(0), keepdims = True)
# playlist_features_std = np.std(playlist_features, axis=(0), keepdims = True)
# playlist_features = (playlist_features - playlist_features_mean) / playlist_features_std
# playlist_features.shape

In [5]:
# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# model.summary()

In [6]:
# read in labels
playlist_tags = []
with open('labels_100_playlists.json') as json_file:
    raw_label = json.load(json_file)
    for i in range(len(raw_label)):
        # filter labels with valid features
        if i in valid_data_indices:
            label = raw_label[i]
            label = np.array(list(label.values()))
#             print(label)
#             label = label.reshape((-1, 1))
            playlist_tags.append(label)

    playlist_tags = np.array(playlist_tags)
#     print(playlist_tags.shape)

In [7]:
# train-test split get random indices
random.seed( 209 )
test_indices = random.sample(range(0, 66), 7)
print(test_indices)

# split features
train_features = []
test_features = []
for i in range(len(playlist_features)):
    if i in test_indices:
        test_features.append(playlist_features[i])
    else:
        train_features.append(playlist_features[i])

train_features = np.array(train_features)
test_features = np.array(test_features)

# split labels
train_labels = []
test_labels = []
for i in range(len(playlist_tags)):
    if i in test_indices:
        test_labels.append(playlist_tags[i])
    else:
        train_labels.append(playlist_tags[i])

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

[2, 21, 36, 5, 41, 43, 11]


In [9]:
input_shape = train_features[0].shape
num_category = train_labels[0].shape[0]

##model building
model = models.Sequential()
#convolutional layer with rectified linear unit activation
model.add(layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
#32 convolution filters used each of size 3x3
#again
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#64 convolution filters used each of size 3x3
#choose the best features via pooling
model.add(layers.MaxPooling2D(pool_size=(3, 3)))
#randomly turn neurons on and off to improve convergence
# model.add(layers.Dropout(0.25))
#flatten since too many dimensions, we only want a classification output
model.add(layers.Flatten())
#fully connected to get all relevant data
model.add(layers.Dense(128, activation='relu'))
#one more dropout for convergence' sake :) 
model.add(layers.Dropout(0.2))
#output a softmax to squash the matrix into output probabilities
model.add(layers.Dense(num_category, activation='sigmoid'))

In [10]:
# # build CNN structure
# input_shape = train_features[0].shape
# num_category = train_labels[0].shape[0]

# # model building
# model = models.Sequential()
# #convolutional layer with rectified linear unit activation
# model.add(layers.Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=input_shape))
# #32 convolution filters with each size 3x3
# # model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
# #64 convolution filters used each of size 3x3
# #choose the best features by adding a pooling layer
# # model.add(layers.MaxPooling2D(pool_size=(3, 3)))
# #randomly turn neurons on and off to improve convergence
# model.add(layers.Dropout(0.25))
# # model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# #64 convolution filters used each of size 3x3
# #choose the best features by adding a pooling layer
# # model.add(layers.MaxPooling2D(pool_size=(3, 3)))
# model.add(layers.Dropout(0.25))
# # model.add(layers.Conv2D(256, (3, 3), activation='relu'))
# # model.add(layers.Conv2D(32, (3, 3), activation='relu'))
# #64 convolution filters used each of size 3x3
# #choose the best features by adding a pooling layer
# # model.add(layers.MaxPooling2D(pool_size=(3, 3)))
# # model.add(layers.Dropout(0.25))
# #flatten layer
# model.add(layers.Flatten())
# #fully connected to get all relevant data
# model.add(layers.Dense(128, activation='relu'))
# #one more dropout for convergence
# # model.add(layers.Dropout(0.5))
# #output multiple sigmoid to generate output probabilities
# model.add(layers.Dense(num_category, activation='sigmoid'))

# model.summary()

In [12]:
# Use adaDelta as the adaptive learning rate
model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy']
             )

In [14]:
# Train Model
# too few data to have a batch_size
batch_size = 5
num_epoch = 30
#model training
model_log = model.fit(train_features, train_labels,
          epochs=num_epoch,
          verbose=1,
          validation_data=(test_features, test_labels))

Train on 59 samples, validate on 7 samples
Epoch 1/30
59/59 [==============================] - 0s 881us/sample - loss: 0.0477 - accuracy: 0.4853 - val_loss: 0.0388 - val_accuracy: 0.4896
Epoch 2/30
59/59 [==============================] - 0s 814us/sample - loss: 0.0469 - accuracy: 0.4853 - val_loss: 0.0389 - val_accuracy: 0.4896
Epoch 3/30
59/59 [==============================] - 0s 832us/sample - loss: 0.0467 - accuracy: 0.4853 - val_loss: 0.0389 - val_accuracy: 0.4896
Epoch 4/30
59/59 [==============================] - 0s 787us/sample - loss: 0.0484 - accuracy: 0.4853 - val_loss: 0.0387 - val_accuracy: 0.4896
Epoch 5/30
59/59 [==============================] - 0s 770us/sample - loss: 0.0473 - accuracy: 0.4853 - val_loss: 0.0385 - val_accuracy: 0.4896
Epoch 6/30
59/59 [==============================] - 0s 723us/sample - loss: 0.0472 - accuracy: 0.4853 - val_loss: 0.0383 - val_accuracy: 0.4896
Epoch 7/30
59/59 [==============================] - 0s 720us/sample - loss: 0.0467 - accuracy

In [15]:
score = model.evaluate(train_features, train_labels, verbose=1)
print(score)
# print('Train loss:', score[0]) 
# print('Train accuracy:', score[1]) 

59/1 [==================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [16]:
# import sys
# np.set_printoptions(threshold=sys.maxsize)
model.predict(train_features)
test_predicted = model.predict(test_features)
test_predicted
# test_predicted = np.exp(test_predicted)
# # test_softmax_sum = np.sum(test_predicted, axis = 1, keepdims=True)
# # test_predicted = test_predicted / test_softmax_sum
# test_predicted

array([[0.1033814 , 0.19981924, 0.07360041, ..., 0.00245315, 0.00193462,
        0.00180954],
       [0.10080591, 0.15980485, 0.06307247, ..., 0.00163779, 0.00133416,
        0.00131047],
       [0.10593888, 0.20870295, 0.07183036, ..., 0.00168559, 0.00161546,
        0.00134528],
       ...,
       [0.13514945, 0.26793402, 0.11710921, ..., 0.0073657 , 0.00680143,
        0.00610012],
       [0.09436324, 0.14985222, 0.05496398, ..., 0.00113788, 0.00103033,
        0.00079548],
       [0.08890319, 0.12798741, 0.05039504, ..., 0.00094579, 0.00078535,
        0.00067958]], dtype=float32)

In [17]:
score = model.evaluate(test_features, test_labels, verbose=1)
print(score)
# print('Test loss:', score[0]) 
# print('Test accuracy:', score[1]) 

7/1 [==================================================================================================================================================================================================================] - 0s 600us/sample - loss: 0.0376 - accuracy: 0.4896
[0.03755860775709152, 0.4895725]


In [18]:
# KNN Model
# ks = [1, 2, 3, 4, 5, 7, 9, 10, 15, 25]
# for k in ks:
#     knn = KNeighborsClassifier(n_neighbors=k)
#     knn.fit(train_features, train_labels)
#     cur_score = np.mean(cross_val_score(knn, train_features, train_labels, cv=5))

#     if cur_score > max_score:
#         max_score = cur_score
#         max_k = k

# print()

In [19]:
### KNN Prediction part ###
### Get data ###
with open('random_playlists.json') as json_file:
    random_playlists = json.load(json_file)

print(len(random_playlists))

10000


In [20]:
# random.seed(209)
playlist_indices = random.sample(range(0, 10000), 10)
print(playlist_indices)

[1276, 5987, 2690, 3677, 6878, 5240, 2816, 3322, 4631, 1117]


In [21]:
# randomely select 10 playlists out of 10000. 
random_playlists_for_recomd = []
for ind in playlist_indices:
    random_playlists_for_recomd.append(random_playlists[ind])
# print (random_playlists_for_recomd)

In [22]:
# extract songs from the playlists
# result random_songs is a list of songs (each element is a pair of artist name and track name)
random_songs = []
for playlist in random_playlists_for_recomd:
    artists = list(playlist["artist_name"].values())
    tracks = list(playlist["track_name"].values())
    song_pairs = list(zip(artists, tracks))
    random_songs = random_songs + song_pairs
    
print(len(random_songs))

407


In [23]:
# list(songs_with_tags.keys())[0][0]

In [24]:
# # get tags for all randomly selected candidate songs
# API_KEY = "caba40286bcee1b73988cac3d1d408e6"
# API_SECRET = "297babeacf457cb7d2bf3bd1584e54ea"
# password_hash = "123456!abcd"
# username = "bootownine"
# network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET,
#                                username=username, password_hash=password_hash)

# songs_with_tags = {}

# #unique tags threshold
# threshold = 3000
# #threshold = 5000
# unique_tags = []
# with open("lastfm_unique_tags.txt", 'r') as f:
#     for line in f:
#         tag_freq = line.split('\t')
#         if int(tag_freq[1]) > threshold:
#             unique_tags.append(tag_freq[0])
#         else:
#             break

# def get_zero_dict():
#     zeros_dict = {}
#     for tag in unique_tags:
#         zeros_dict.update({tag: 0.0})
#     return zeros_dict

# def get_tags_from_track(artist_name, track_name):
#     try:
#         track = network.get_track(artist_name, track_name)
#         tags = track.get_top_tags()
#         tags_with_weights = []
#         for tag_item in tags:
#             if str(tag_item.item) in unique_tags:
#                 tags_with_weights.append((str(tag_item.item), int(tag_item.weight)))
#         return tags_with_weights
#     except:
#         return None

# for track in random_songs:
#     tag_dict = get_zero_dict()
#     tags_with_weights = get_tags_from_track(track[0], track[1])
#     if tags_with_weights is not None and len(tags_with_weights) != 0:
#         for pair in tags_with_weights:
#             tag_dict.update({pair[0]: pair[1]/100.0})
# #     print(tag_dict)
#     songs_with_tags.update({(track[0], track[1]): list(tag_dict.values())})

In [25]:
candidates_with_tags = []
with open('candidates_with_tags.json') as json_file:
    candidates_with_tags = json.load(json_file)
#     print(candidates_with_tags.keys())

In [26]:
# INPUT: tags for candidate songs, and the predicted tags of one playlist in the test dataset from a model
# OUTPUT: a song recommendation
def make_recommendation(candidate_tags, playlist_predicted_tag):    
    min_distance = float('inf');
    rec = -1;
    
    candidate_songs = list(candidate_tags.keys())
    for c in range(len(candidate_songs)):
        key = candidate_songs[c]
        candidate = candidate_tags[key]
        each_distance = np.sum((np.array(candidate) - np.array(playlist_predicted_tag)) ** 2)
        if each_distance < min_distance:
            min_distance = each_distance;
#             print(min_distance)
            rec = key;
    return rec;

In [27]:
### Predict by KNN ### 
candidate_tags = candidates_with_tags
recommendations = []
for each_playlist_predicted_tag in test_predicted:
    rec_for_test = make_recommendation(candidate_tags, each_playlist_predicted_tag)
    recommendations.append(rec_for_test)


In [28]:
print(recommendations)

['Disclosure*,*Omen - Radio Edit', 'Disclosure*,*Omen - Radio Edit', 'Disclosure*,*Omen - Radio Edit', 'Disclosure*,*Omen - Radio Edit', 'Ester Dean*,*Drop It Low', 'Disclosure*,*Omen - Radio Edit', 'Disclosure*,*Omen - Radio Edit']


In [48]:
# songs_recommended = [];
# for rec in recommendations:
#     songs_recommended.append(random_songs[rec])
# print(songs_recommended)